<a href="https://colab.research.google.com/github/yunjiyunji/ML/blob/main/pred_houseprice/1_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **부산 남구 아파트 가격 예측**
- 국토교통부에서 제공하는 주택 매매 실거래가 데이터 활용 http://rtdown.molit.go.kr/
- 2017-2021 아파트 매매 실거래가 데이터를 사용해 linear regression model 생성
- 2022 아파트 매매 가격을 예측하고 실제 데이터와 비교

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# train.csv - 부산 남구 아파트 매매 실거래가 2017-2021 데이터
df = pd.read_csv('/content/drive/MyDrive/wap AI/train.csv', encoding='cp949', thousands = ',')

### Linear Regression - Gradient Descent

In [4]:
# 데이터 - 숫자형 변수를 가지는 컬럼
xy = df[['계약년월', '계약일','전용면적(㎡)', '층', '건축년도','거래금액(만원)']].values

# 텐서 형태로
x = torch.FloatTensor(xy[:, 0:-1])
y = torch.FloatTensor(xy[:, [-1]])

# 모델 선언 및 초기화
model = nn.Linear(5,1)

# optimizer 설정 - 경사하강법
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

# 에포크 반복
for epoch in range(100):

  # H(x) 계산 (모델의 예측값)
  prediction = model(x)

  # cost 계산 (MSE 손실 계산)
  cost = F.mse_loss(prediction, y)

  # cost로 H(x) 개선
  # gradient를 0으로 초기화
  optimizer.zero_grad()
  # 비용함수를 미분하여 gradient 계산
  cost.backward() # 오차 역전파
  # W와 b 업데이트 (최적화 진행)
  optimizer.step()

  if epoch % 10 == 0:
    print(f'epoch{epoch} cost:{cost.item()}')

epoch0 cost:8187419648.0
epoch10 cost:nan
epoch20 cost:nan
epoch30 cost:nan
epoch40 cost:nan
epoch50 cost:nan
epoch60 cost:nan
epoch70 cost:nan
epoch80 cost:nan
epoch90 cost:nan




> ### 비용함수 nan인 이유
 1) gradient의 값이 너무 커서 weight 업데이트 시 발산되어 weight가 nan이 됨
- learning rate 를 작게 하여 시도해보았으나 문제가 해결되지 않음
>
>2) **확률적 경사하강법(SGD)의 한계**
>
>- **다른 최적화 알고리즘** 탐색
>
>3) 데이터 정규화

### Linear Regression - Adam optimizer

In [5]:
# 데이터 - 숫자형 변수를 가지는 컬럼
xy = df[['계약년월', '계약일','전용면적(㎡)', '층', '건축년도','거래금액(만원)']].values

# 텐서 형태로
x = torch.FloatTensor(xy[:, 0:-1])
y = torch.FloatTensor(xy[:, [-1]])

# 모델 선언 및 초기화
model = nn.Linear(5,1)

# optimizer 설정 - Adam
optimizer = torch.optim.Adam(model.parameters(),lr = 0.1)

# 에포크 반복
for epoch in range(1000000):

  # H(x) 계산 (모델의 예측값)
  prediction = model(x)

  # cost 계산 (MSE 손실 계산)
  cost = F.mse_loss(prediction, y)

  # cost로 H(x) 개선
  # gradient를 0으로 초기화
  optimizer.zero_grad()
  # 비용함수를 미분하여 gradient 계산
  cost.backward() # 오차 역전파
  # W와 b 업데이트 (최적화 진행)
  optimizer.step()

  if epoch % 100000 == 0:
    print(f'epoch{epoch} cost:{cost.item()}')

epoch0 cost:14410732544.0
epoch100000 cost:305517664.0
epoch200000 cost:295444512.0
epoch300000 cost:293474464.0
epoch400000 cost:293084640.0
epoch500000 cost:293008256.0
epoch600000 cost:292992416.0
epoch700000 cost:292988992.0
epoch800000 cost:292988256.0
epoch900000 cost:293086880.0


### 2022 데이터 예측

In [6]:
# test.csv - 부산 남구 아파트 매매 실거래가 2022 데이터 (10월13일까지 데이터)
test_df = pd.read_csv('/content/drive/MyDrive/wap AI/test.csv', encoding='cp949', thousands = ',')
test_xy = test_df[['계약년월', '계약일','전용면적(㎡)', '층', '건축년도','거래금액(만원)']].values
x_test = torch.FloatTensor(test_xy[:,0:-1])
y_test = torch.FloatTensor(test_xy[:,[-1]])

In [7]:
y_pred = model(x_test)

In [8]:
for i in range(10):
  print('예측된 가격:',int(y_pred[i].item()), '실제 거래금액:', y_test[i].item())

예측된 가격: 19812 실제 거래금액: 16200.0
예측된 가격: 16298 실제 거래금액: 16300.0
예측된 가격: 16696 실제 거래금액: 16500.0
예측된 가격: 16352 실제 거래금액: 17250.0
예측된 가격: 22696 실제 거래금액: 17500.0
예측된 가격: 23093 실제 거래금액: 17500.0
예측된 가격: 22773 실제 거래금액: 22500.0
예측된 가격: 16721 실제 거래금액: 18700.0
예측된 가격: 22428 실제 거래금액: 18000.0
예측된 가격: 31903 실제 거래금액: 25000.0
